<a href="https://colab.research.google.com/github/kinjaljoshi/s2t-training/blob/main/react_lora_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#connect to storage bucket
from google.colab import auth
auth.authenticate_user()
PROJECT_ID = 'llm-text-to-sql-445914'
!gcloud config set project $PROJECT_ID

In [ ]:
import json

# Load the JSON file
try:
    with open("/content/reasoning_refined_prompts_with_addition.json", "r") as file:
        data = json.load(file)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    exit()

# Traverse the JSON to locate the problematic key
def find_missing_key(data, key):
    if isinstance(data, dict):
        if key in data:
            return
        else:
            print(f"Key '{key}' not found in: {data}")
    elif isinstance(data, list):
        for index, item in enumerate(data):
            try:
                find_missing_key(item, key)
            except Exception as e:
                print(f"Error in item {index}: {e}")

# Specify the key to search for
find_missing_key(data, "previous_action")


In [ ]:
#update the prompts JSON with new text
import json


with open("/content/reasoning_refined_prompts_with_addition.json", "r") as file:
    input_json = json.load(file)

# formatted_prompts = []
# for entry in input_json:
#   if 'Retrieve' in entry['sql_prompt']:
#         entry['sql_prompt'] = entry['sql_prompt'].replace('Retrieve', 'Find')

#   entry['sql_prompt'] = f"****Read and Understand the given question and given table or tabless and column descriptions and Write a SQL query and always use miminum 3 character aliase based on the given schema and reasoning to {entry['sql_prompt']}"

  prompt = f"""
Input: {entry['sql_prompt']}
{entry['sql_context']}

Reasoning:
{entry['reasoning']}

Constraints:
{entry['previous_action']}

Final Answer:
"""
  formatted_prompts.append(prompt)


In [ ]:
len(formatted_prompts)

2069

In [ ]:
inputs = [prompt for prompt in formatted_prompts]
labels = [entry['previous_action'] for entry in input_json]


In [ ]:
inputs[0]

'\nInput: **Write a SQL query and always use miminum 3 character aliase based on the given schema and reasoning to Find customer names and their corresponding total order amounts.\n TABLE STT_CUSTOMERS \'Contains customer details\' ( CUSTOMER_ID \'Unique identifier for a customer\', CUSTOMER_NAME \'Name of the customer\', EMAIL \'Email address of the customer\' ); TABLE STT_ORDERS \'Contains order details\' ( ORDER_ID \'Unique identifier for an order\', CUSTOMER_ID \'Foreign key linking to customers\', ORDER_DATE \'Date when the order was placed\', TOTAL_AMOUNT \'Total monetary value of the order\' ) \'FOREIGN_KEY CUSTOMER_ID to STT_CUSTOMERS(CUSTOMER_ID)\'; \n\nReasoning:\nTo solve the SQL prompt: "Retrieve customer names and their corresponding total order amounts.", the following steps are taken:\n\n1. **Analyze the Question and Column Descriptions to Identify Relevant Columns and Calculations:**\n   - From the provided context, the following columns are identified based on their de

In [ ]:
from sklearn.model_selection import train_test_split

# Split inputs and labels into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    inputs, labels, test_size=0.2, random_state=42
)

In [ ]:
len(train_inputs), len(train_labels), len(val_inputs), len(val_labels)

(1655, 1655, 414, 414)

In [ ]:
import nltk
nltk.download('punkt', force=True)
nltk.download('punkt_tab', force=True)
print(nltk.find('tokenizers/punkt'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


/root/nltk_data/tokenizers/punkt


In [ ]:
from nltk.tokenize import sent_tokenize
from transformers import T5Tokenizer

# Configure the tokenizer's pad_token to be the eos_token
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token

def tokenize_with_sliding_window(inputs, labels, max_length=512, stride=256):
    tokenized_inputs = []
    tokenized_labels = []

    for input_text, label_text in zip(inputs, labels):
        # Sentence tokenization
        sentences = sent_tokenize(input_text)
        sentence_tokens = []

        # Tokenize each sentence and combine
        for sentence in sentences:
            tokens = tokenizer(sentence, truncation=False, padding=False)["input_ids"]
            sentence_tokens.extend(tokens)

        # Create sliding windows for input tokens
        for i in range(0, len(sentence_tokens), stride):
            window = sentence_tokens[i:i + max_length]
            # Ensure the window does not exceed max_length
            if len(window) > max_length:
                window = window[:max_length]
            padded_window = tokenizer.pad({'input_ids': [window]}, max_length=max_length, padding=True)["input_ids"]
            tokenized_inputs.append(padded_window[0])  # Extract the tokenized and padded input

        # Tokenize the label text
        # Repeat the label for each sliding window
            label_tokens = tokenizer(
                label_text, truncation=True, padding="max_length", max_length=max_length)["input_ids"]

            # Replace padding tokens in labels with -100
            label_tokens = [
                token if token != tokenizer.pad_token_id else -100 for token in label_tokens
            ]
            tokenized_labels.append(label_tokens)

    return tokenized_inputs, tokenized_labels

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
tokenized_inputs, tokenized_labels = tokenize_with_sliding_window(train_inputs, train_labels)
tokenized_inputs_val, tokenized_labels_val = tokenize_with_sliding_window(val_inputs, val_labels)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(tokenized_inputs), len(tokenized_labels), len(tokenized_inputs_val), len(tokenized_labels_val)

(4673, 4673, 1159, 1159)

In [ ]:
!gsutil -m cp -r gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq /content/

Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/README.md...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/adapter_config.json...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/adapter_model.safetensors...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/added_tokens.json...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/special_tokens_map.json...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/spiece.model...
Copying gs://llm-pipeline-resources/models/flan-t5-large-peft-sttq/tokenizer_config.json...
| [7/7 files][ 18.8 MiB/ 18.8 MiB] 100% Done                                    
Operation completed over 7 objects/18.8 MiB.                                     


In [ ]:
#load merged model
!gsutil -m cp -r gs://llm-pipeline-resources/models/peft_morged_flan_sttq /content/

Copying gs://llm-pipeline-resources/models/peft_morged_flan_sttq/config.json...
Copying gs://llm-pipeline-resources/models/peft_morged_flan_sttq/generation_config.json...
Copying gs://llm-pipeline-resources/models/peft_morged_flan_sttq/model.safetensors...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

| [3/3 files][  2.9 GiB/  2.9 GiB] 100% Done 129.1 MiB/s ETA 00:00:00           
Operation completed over 3 objects/2.9 GiB.                                      


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import PeftModel

merged_model = T5ForConditionalGeneration.from_pretrained("/content/peft_morged_flan_sttq").cuda()
trainable_params = [name for name, param in merged_model.named_parameters() if param.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")


Number of trainable parameters: 558


In [ ]:
trainable_layers = []

for name, param in merged_model.named_parameters():
    if param.requires_grad:  # Check if the parameter is trainable
        trainable_layers.append(name)  # Add layer name to the list

# Print the layers with trainable parameters
print("Trainable Layers:")
for layer in trainable_layers:
    print(layer)


Trainable Layers:
shared.weight
encoder.block.0.layer.0.SelfAttention.q.weight
encoder.block.0.layer.0.SelfAttention.k.weight
encoder.block.0.layer.0.SelfAttention.v.weight
encoder.block.0.layer.0.SelfAttention.o.weight
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight
encoder.block.0.layer.0.layer_norm.weight
encoder.block.0.layer.1.DenseReluDense.wi_0.weight
encoder.block.0.layer.1.DenseReluDense.wi_1.weight
encoder.block.0.layer.1.DenseReluDense.wo.weight
encoder.block.0.layer.1.layer_norm.weight
encoder.block.1.layer.0.SelfAttention.q.weight
encoder.block.1.layer.0.SelfAttention.k.weight
encoder.block.1.layer.0.SelfAttention.v.weight
encoder.block.1.layer.0.SelfAttention.o.weight
encoder.block.1.layer.0.layer_norm.weight
encoder.block.1.layer.1.DenseReluDense.wi_0.weight
encoder.block.1.layer.1.DenseReluDense.wi_1.weight
encoder.block.1.layer.1.DenseReluDense.wo.weight
encoder.block.1.layer.1.layer_norm.weight
encoder.block.2.layer.0.SelfAttention.q.weight
encode

In [ ]:
#unfreeze encoder/decoder layers
for name, param in merged_model.named_parameters():
    if "encoder.block" in name or "decoder.block" in name:
        # Extract layer index
        layer_idx = int(name.split(".")[2])
        # Unfreeze last 4 encoder and decoder layers
        if layer_idx >= merged_model.config.num_layers - 4:
            param.requires_grad = True
        else:
            param.requires_grad = False
    else:
        param.requires_grad = False  # Freeze all other layers



In [ ]:
trainable_params = [name for name, param in merged_model.named_parameters() if param.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")


Number of trainable parameters: 92


In [ ]:
#unfreeze decoder layers
for name, param in merged_model.named_parameters():
    if "decoder.block" in name:
        layer_idx = int(name.split(".")[2])  # Extract the layer index
        if layer_idx >= merged_model.config.num_hidden_layers - 6:  # Unfreeze top 6 layers
            param.requires_grad = True
        else:
            param.requires_grad = False


In [ ]:
#unfreeze cross attention layers
for name, param in merged_model.named_parameters():
    if "decoder.block" in name and "EncDecAttention" in name:
        param.requires_grad = True


In [ ]:
#unfreeze liner layers
for name, param in merged_model.named_parameters():
    if "lm_head" in name:  # This is typically the final linear layer
        param.requires_grad = True


In [ ]:
trainable_params = [name for name, param in merged_model.named_parameters() if param.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")


Number of trainable parameters: 558


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from peft import PeftModel

base_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

for param in base_model.parameters():
    param.requires_grad = False

peft_model = PeftModel.from_pretrained(base_model, "/content/flan-t5-large-peft-sttq", is_trainable=True )
trainable_params = [name for name, param in peft_model.named_parameters() if param.requires_grad]
print(f"Number of trainable parameters: {len(trainable_params)}")
#print("Trainable parameters:", trainable_params)


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Number of trainable parameters: 288


In [ ]:
peft_model.to("cuda")

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 1024)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 1024)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.3, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
       

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
import random

# Define a custom dataset class
class SQLDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.inputs[idx], device="cuda"),
            "attention_mask": torch.tensor([1] * len(self.inputs[idx]), device="cuda"),
            "labels": torch.tensor(self.labels[idx], device="cuda")
        }



In [ ]:
# from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer
# from torch.utils.data import Dataset
# import torch
# from sklearn.model_selection import train_test_split
# import random



# # Split the data into training and validation sets
# def split_dataset(inputs, labels, test_size=0.2, random_seed=42):
#     combined = list(zip(inputs, labels))
#     random.seed(random_seed)
#     random.shuffle(combined)  # Shuffle for a balanced mix
#     inputs, labels = zip(*combined)  # Unzip after shuffle
#     train_inputs, val_inputs, train_labels, val_labels = train_test_split(
#         inputs, labels, test_size=test_size, random_state=random_seed
#     )
#     return train_inputs, val_inputs, train_labels, val_labels

# # # Split the tokenized inputs and labels
# # train_inputs, val_inputs, train_labels, val_labels = split_dataset(tokenized_inputs, tokenized_labels)

# # # Prepare datasets
# # train_dataset = SQLDataset(train_inputs, train_labels)
# # val_dataset = SQLDataset(val_inputs, val_labels)


In [ ]:
train_dataset = SQLDataset(tokenized_inputs, tokenized_labels)
val_dataset = SQLDataset(tokenized_inputs_val, tokenized_labels_val)

In [ ]:
# from sklearn.model_selection import train_test_split

# def group_and_split_dataset(inputs, tokenized_inputs, tokenized_labels, test_size=0.2, random_seed=42):
#     """
#     Ensures all sliding windows and repeated labels from the same input remain in the same split.

#     Args:
#         inputs: List of original prompts (before tokenization).
#         tokenized_inputs: List of tokenized sliding windows (derived from inputs).
#         tokenized_labels: List of labels repeated for each sliding window.
#         test_size: Fraction of data to reserve for validation.
#         random_seed: Random seed for reproducibility.

#     Returns:
#         train_inputs, val_inputs, train_labels, val_labels
#     """
#     # Group tokenized inputs and labels by their original index
#     grouped_data = []
#     idx = 0
#     for i, original_input in enumerate(inputs):
#         group_windows = []
#         group_labels = []
#         while idx < len(tokenized_inputs) and idx < len(tokenized_labels):
#             group_windows.append(tokenized_inputs[idx])
#             group_labels.append(tokenized_labels[idx])
#             idx += 1
#         grouped_data.append((group_windows, group_labels))

#     # Split groups into training and validation sets
#     train_groups, val_groups = train_test_split(
#         grouped_data, test_size=test_size, random_state=random_seed
#     )

#     # Flatten grouped data back into sliding windows and labels
#     train_inputs, train_labels = zip(*[(window, label) for group in train_groups for window, label in zip(*group)])
#     val_inputs, val_labels = zip(*[(window, label) for group in val_groups for window, label in zip(*group)])

#     return list(train_inputs), list(val_inputs), list(train_labels), list(val_labels)



In [ ]:
# train_inputs, val_inputs, train_labels, val_labels = group_and_split_dataset(
#     inputs, tokenized_inputs, tokenized_labels, test_size=0.2
# )


In [ ]:
len(train_dataset), len(val_dataset)

(4673, 1159)

In [ ]:
import os

# Disable wandb
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2)

In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=50,
    fp16=False,
    load_best_model_at_end=True
)

# Define the Trainer
# Define the Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,       # Ensure padding and tokenization consistency
    callbacks=[early_stopping]
)


# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-6eaad8f21522>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.203900,0.112502
2,0.129200,0.091719
3,0.141100,0.081744
4,0.098300,0.075375
5,0.107400,0.071317
6,0.107700,0.068257
7,0.084400,0.064966
8,0.099800,0.063612
9,0.084600,0.061376
10,0.070300,0.061659


TrainOutput(global_step=22980, training_loss=0.09728372884481445, metrics={'train_runtime': 16591.0141, 'train_samples_per_second': 5.538, 'train_steps_per_second': 1.385, 'total_flos': 2.0106589166958182e+17, 'train_loss': 0.09728372884481445, 'epoch': 20.0})

In [ ]:

# Save the fine-tuned model
peft_model.save_pretrained("/content/model/fine_tuned_model_v1.5")


In [ ]:
!gsutil -m cp -r /content/model/fine_tuned_model_v1.5 gs://llm-pipeline-resources/models

Copying file:///content/model/fine_tuned_model_v1.5/adapter_config.json [Content-Type=application/json]...
Copying file:///content/model/fine_tuned_model_v1.5/adapter_model.safetensors [Content-Type=application/octet-stream]...
Copying file:///content/model/fine_tuned_model_v1.5/README.md [Content-Type=text/markdown]...
-
Operation completed over 3 objects/18.0 MiB.                                     


# **Run Training on Merged Model**

In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=50,
    fp16=False,
    load_best_model_at_end=True
)

# Define the Trainer
# Define the Trainer
trainer = Trainer(
    model=merged_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Add validation dataset
    tokenizer=tokenizer,       # Ensure padding and tokenization consistency
    callbacks=[early_stopping]
)


# Train the model
trainer.train()

merged_model.save_pretrained("/content/model/merged_react_v1.1")


!gsutil -m cp -r /content/model/merged_react_v1.1 gs://llm-pipeline-resources/models

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-c8a91b539c20>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.085800,0.065637
2,0.055900,0.055605
3,0.044800,0.051245
4,0.049000,0.050638
5,0.029500,0.051097


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Copying file:///content/model/merged_react_v1.1/generation_config.json [Content-Type=application/json]...
Copying file:///content/model/merged_react_v1.1/config.json [Content-Type=application/json]...
Copying file:///content/model/merged_react_v1.1/model.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [3/3 files][

In [ ]:
merged_model.save_pretrained("/content/model/merged_react_v1.2", safe_serialization=True)

In [ ]:
!gsutil -m cp -r /content/model/merged_react_v1.2 gs://llm-pipeline-resources/models

Copying file:///content/model/merged_react_v1.2/generation_config.json [Content-Type=application/json]...
Copying file:///content/model/merged_react_v1.2/config.json [Content-Type=application/json]...
Copying file:///content/model/merged_react_v1.2/model.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [3/3 files][